In [1]:
!pip install anomalib[full]

In [7]:
from typing import Any
import time
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
import pandas as pd

from anomalib.data import MVTecAD
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import EfficientAd
from anomalib.utils.visualization import ImageResult

In [8]:
datasets = ["mvtec"]
categories = {
    "mvtec": ["bottle", "cable"],
    "2": ["capsule", "carpet", "grid", "hazelnut", "leather",
                "metal_nut", "pill", "screw", "tile", "toothbrush", "transistor", "wood", "zipper"],
    "visa": ["candle", "capsules", "cashew", "chewinggum", "fryum", "macaroni1", "macaroni2",
            "pcb1", "pcb2", "pcb3", "pcb4", "pipe_fryum"]
}

teacher_out_channels_list = [128, 512]
model_sizes = ["small"]
learning_rates = [1e-5, 1e-3]
weight_decays = [1e-6, 1e-4]

results = []

for dataset in datasets:
    print(f"\nEvaluating on {dataset} dataset...")

    for category in categories["mvtec"]:
        for model_size in model_sizes:
            for lr in learning_rates:
                for weight_decay in weight_decays:
                    print(f"\nParameters: model_size={model_size}, lr={lr}, weight_decay={weight_decay}")

                    datamodule = MVTecAD(
                        root = './datasets/MVTecAD',
                        category=category,
                        train_batch_size=1,
                        eval_batch_size=1,
                        num_workers=0)

                    datamodule.prepare_data()
                    datamodule.setup()

                    model = EfficientAd(
                        model_size=model_size,
                        lr=lr,
                        weight_decay=weight_decay,
                    )

                    engine = Engine(
                        max_epochs = 10,
                        accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
                        devices=1,
                        logger=False,
                    )

                     # Training
                    start_time = time.time()
                    engine.fit(model=model, datamodule=datamodule)
                    training_time = time.time() - start_time

                    # Testing
                    start_time = time.time()
                    test_result = engine.test(model=model, datamodule=datamodule)
                    inference_time = time.time() - start_time


                    results.append({
                        "dataset": dataset,
                        "category" : category,
                        "model_size": model_size,
                        "lr": lr,
                        "weight_decay": weight_decay,
                        "image_AUROC": test_result[0]['image_AUROC'],
                        "image_F1Score": test_result[0]['image_F1Score'],
                        "pixel_AUROC": test_result[0]['pixel_AUROC'],
                        "pixel_F1Score": test_result[0]['pixel_F1Score'],
                        "training_time": training_time,
                        "inference_time": inference_time,
                        "latency": inference_time / len(datamodule.test_dataloader().dataset)})


results_df = pd.DataFrame(results)


Evaluating on mvtec dataset...

Parameters: model_size=small, lr=1e-05, weight_decay=1e-06


INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | pre_processor  | PreProcessor     | 0      | train
1 | post_processor | PostProcessor    | 0      | train
2 | evaluator      | Evaluator        | 0      | train
3 | model          | EfficientAdModel | 8.1 M  | train
------------------------------------------------------------
8.1 M     Trainable params
0         Non-trainable params
8.1 M     Total params
32.235    Total estimated model params size (MB)
48        Modules in train mode
7         Modules in eval mode
INFO:lightning.pytorch.callbac

Training: |          | 0/? [00:00<?, ?it/s]


efficientad_pretrained_weights.zip: 0.00B [00:00, ?B/s]
efficientad_pretrained_weights.zip:   0%|          | 8.19k/40.0M [00:00<1:09:59, 9.51kB/s]
efficientad_pretrained_weights.zip:  26%|██▋       | 10.5M/40.0M [00:00<00:02, 14.6MB/s]  
efficientad_pretrained_weights.zip:  47%|████▋     | 18.9M/40.0M [00:01<00:00, 25.9MB/s]
efficientad_pretrained_weights.zip:  59%|█████▉    | 23.7M/40.0M [00:01<00:00, 30.1MB/s]
efficientad_pretrained_weights.zip:  79%|███████▊  | 31.5M/40.0M [00:01<00:00, 35.7MB/s]
efficientad_pretrained_weights.zip: 40.0MB [00:01, 27.5MB/s]                            
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(

imagenette2.tgz: 0.00B [00:00, ?B/s]
imagenette2.tgz:   0%|          | 8.19

Validation: |          | 0/? [00:00<?, ?it/s]

...

Calculate Validation Dataset Quantiles: 100%|██████████| 150/150 [00:11<00:00, 12.84it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 150/150 [00:11<00:00, 12.94it/s]
INFO:lightning_fabric.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning_fabric.utilities.rank_zero:The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.8738754987716675     │
│       image_F1Score       │    0.8444444537162781     │
│        pixel_AUROC        │    0.9626357555389404     │
│       pixel_F1Score       │    0.5844300389289856     │
└───────────────────────────┴───────────────────────────┘

In [9]:
 # Save results
results_df.to_csv("efficientad_parameter_study_results.csv", index=False)
# Print summary statistics
print("\nSummary Statistics:")
print("\nBy Dataset:")
print(results_df.groupby("dataset")[["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]].mean())
print("\nBy Category:")
print(results_df.groupby(["dataset", "category"])[["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]].mean())
print("\nBy Model Size:")
print(results_df.groupby("model_size")[["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]].mean())
print("\nBy Learning Rate:")
print(results_df.groupby("lr")[["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]].mean())
print("\nBy Weight Decay:")
print(results_df.groupby("weight_decay")[["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]].mean())
# Find best parameters for each metric
print("\nBest Parameters:")
for metric in ["image_AUROC", "image_F1Score", "pixel_AUROC", "pixel_F1Score", "latency"]:
    best_idx = results_df[metric].idxmax() if metric != "latency" else results_df[metric].idxmin()
    print(f"\nBest {metric}:")
    print(results_df.iloc[best_idx])


Summary Statistics:

By Dataset:
         image_AUROC  image_F1Score  pixel_AUROC  pixel_F1Score   latency
dataset                                                                  
mvtec       0.894482       0.897952     0.886269       0.522318  0.218002

By Category:
                  image_AUROC  image_F1Score  pixel_AUROC  pixel_F1Score  \
dataset category                                                           
mvtec   bottle       0.984921       0.973933     0.894689       0.637178   
        cable        0.804043       0.821972     0.877848       0.407457   

                   latency  
dataset category            
mvtec   bottle    0.183245  
        cable     0.252759  

By Model Size:
            image_AUROC  image_F1Score  pixel_AUROC  pixel_F1Score   latency
model_size                                                                  
small          0.894482       0.897952     0.886269       0.522318  0.218002

By Learning Rate:
         image_AUROC  image_F1Score  pixel_

In [10]:
results_df.head(20)

,dataset,category,model_size,lr,weight_decay,image_AUROC,image_F1Score,pixel_AUROC,pixel_F1Score,training_time,inference_time,latency
0,mvtec,bottle,small,0.00001,0.000001,0.965079,0.959350,0.829718,0.523061,552.885983,15.342267,0.184847
1,mvtec,bottle,small,0.00001,0.000100,0.974603,0.952381,0.806084,0.486399,492.771465,15.455026,0.186205
2,mvtec,bottle,small,0.00100,0.000001,1.000000,0.992000,0.971592,0.770051,483.288852,14.860142,0.179038
3,mvtec,bottle,small,0.00100,0.000100,1.000000,0.992000,0.971362,0.769203,485.480160,15.180002,0.182892
4,mvtec,cable,small,0.00001,0.000001,0.715705,0.798122,0.787686,0.207376,744.602702,37.886746,0.252578
5,mvtec,cable,small,0.00001,0.000100,0.736038,0.798030,0.802000,0.257306,753.712569,37.845559,0.252304
6,mvtec,cable,small,0.00100,0.000001,0.890555,0.847291,0.959072,0.580715,741.744297,38.322157,0.255481
7,mvtec,cable,small,0.00100,0.000100,0.873875,0.844444,0.962636,0.584430,741.859037,37.600855,0.250672
